In [14]:
import pandas as pd
import os
import logging
import shutil
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
from sshtunnel import SSHTunnelForwarder


import warnings


DATA = {
    "sales": {
        "FOLDER_PATH_IN": 'C:\\Users\\dmandree\\Downloads\\TL_new',
        "FOLDER_PATH_OUT": 'C:\\Users\\dmandree\\Downloads\\TL_arch',
        "SHEET": 'TurnoverList',
        "COL_NAMES": ['Day', 'Store', 'Company', 'Open', 'Amount', 'Curr', 'Pcs', 'Rcp', 'People', 'Hours', 'Work', 'Comp:', 'Open_1', 'Amount_1', 'Curr_1', 'Pcs_1', 'Rcp_1', 'People_1', 'Hours_1', 'Work_1'],
        "COMPANIES": ['Guess Kazakhstan', 'Guess CIS'],
        "SKIP": 0,
        "IF_EXISTS": 'append'
    },
    "ms_sales": {
        "FOLDER_PATH_IN": 'C:\\Users\\dmandree\\Downloads\\RTL_new',
        "FOLDER_PATH_OUT": 'C:\\Users\\dmandree\\Downloads\\RTL_arch',
        "SHEET": 'RTL50000_by_season_by_store old',
        "COL_NAMES": ['Company', 'Country', 'Day', 'Mfg Season', 'Line Code', 'Gender', 'Dept Group', 'Dept', 'Sub Dept', 'Class', 'Class_1', 'Style', 'Style_1', 'Chain', 'Store', 'Store_1', 'Metrics', 'Ttl Sls Qty', 'TTL Curr Rtl Price €', 'Discount €', 'Ttl Sls €', 'Ttl Cost LC', 'Ttl Sls Trasp Cost LC', 'Ttl Cost €', 'Ttl Sls LC', 'Ttl Sls Trasp Cost €'],
        "COMPANIES": ['RU', 'KZ'],
        "SKIP": 3,
        "IF_EXISTS": 'append'
    },
    "ms_stock": {
        "FOLDER_PATH_IN": 'C:\\Users\\dmandree\\Downloads\\FNC_new',
        "FOLDER_PATH_OUT": 'C:\\Users\\dmandree\\Downloads\\FNC_arch',
        "SHEET": 'FNC03-50001-Margin_stock all st',
        "COL_NAMES": ['Company', 'Day', 'Store', 'Store_1', 'Mfg Season', 'Line Code', 'Line_Code_1', 'Style', 'Style_1', 'Sub_Dept', 'Sub_Dept_1', 'Metrics', 'TTL EOH Ttl Qty', 'TTL Loading Cost €', 'TTL Loading Cost LC', 'TTL Trasp Cost €', 'Cost €'],
        "COMPANIES": ['RU', 'KZ'],
        "SKIP": 2,
        "IF_EXISTS": 'replace'
    }
}


# Folder for Dict
DICT_PATH = 'C:\\Users\\dmandree\\OneDrive - Guess Inc\\D Project\\Dict\\Mapping.xlsx'

# Список страниц, которые мы трансформируем в датафреймы
LIST_OF_SHEETS = ["Stores", "Dist_managers", "VM", "Fin_Calendar_old", "Fin_Calendar_new", "Template", "Start_date"]

# DB and SSH cnnection parameters
DB_PARAMS = {
    'database': 'postgres',
    'user': 'postgres',
    'password': '1296',
    'host': 'localhost'
}

SSH_TUNNEL_PARAMS = {
    'ssh_address_or_host': ('79.174.86.163', 22),
    'ssh_username': 'root',
    'ssh_password': 'S0SJcmYwL0ZsmUId',
    'remote_bind_address': ('127.0.0.1', 5432),
    'local_bind_address': ('127.0.0.1', 8001)
}

# Logging
logging.basicConfig(level=logging.INFO)

# Ignore all UserWarnings
warnings.filterwarnings("ignore", category=UserWarning)

# Function to read Excel files
def read_excel_files(FOLDER_PATH_IN, FOLDER_PATH_OUT, SHEET, SKIP, COL_NAMES):
    file_list = os.listdir(FOLDER_PATH_IN)
    dfs = []
    for file in file_list:
        file_path = os.path.join(FOLDER_PATH_IN, file)
        with pd.ExcelFile(file_path) as xls:
            data = pd.read_excel(xls, sheet_name=SHEET, skiprows=SKIP, names=COL_NAMES)
            dfs.append(data)
        # Moving the file after processing
        # move_processed_file(file_path, FOLDER_PATH_OUT, file)
    
    # Check if the list is not empty    
    if dfs:
        df = pd.concat(dfs, ignore_index=True)
        return df
    else:
        print("No Excel files found or no data read from the files.")
        return None

# Function to move file to archive folder
def move_processed_file(file_path, FOLDER_PATH_OUT, file):
    new_path = os.path.join(FOLDER_PATH_OUT, file)
    if os.path.exists(new_path):
        try:
            os.remove(new_path)
        except PermissionError:
            print("The file is in use by another process and cannot be deleted.")
    try:
        shutil.move(file_path, FOLDER_PATH_OUT)
    except (IOError, shutil.Error) as e:
        print(f"Error while moving file '{file_path}': {e}")

# Function to create dict data
def load_excel_sheets(DICT_PATH, LIST_OF_SHEETS):
    return {sheet: pd.read_excel(DICT_PATH, sheet_name=sheet) for sheet in LIST_OF_SHEETS}

# Function to process data
def process_data(df, COMPANIES):
    if df is None or df.empty:
        return df
    else:
        if df['Day'].dtype == 'O':
            df['Day'] = df['Day'].str[-10:].str.replace(',', '').str.replace(' ', '')
        df["Day"] = pd.to_datetime(df["Day"]).dt.date
        df = df.loc[df['Company'].isin(COMPANIES)]
        df.columns = df.columns.str.lower().str.replace(' ', '_')
        return df

# Function to filtering unique dates in a dataframe
def create_outer_df(df):
    unique_combinations = df['day'].unique()
    outer_df = pd.DataFrame(unique_combinations, columns=['key'])
    return outer_df

# Function for creating an SSH tunnel
def create_ssh_tunnel():
    ssh_tunnel = SSHTunnelForwarder(**SSH_TUNNEL_PARAMS)
    return ssh_tunnel

# Function to connecting to a database
def create_db_engine(ssh_tunnel):
    DB_PARAMS['port'] = ssh_tunnel.local_bind_port
    engine_str = f"postgresql://{DB_PARAMS['user']}:{DB_PARAMS['password']}@{DB_PARAMS['host']}:{DB_PARAMS['port']}/{DB_PARAMS['database']}"
    engine = create_engine(engine_str)
    return engine

# Function to get date intersections
def get_intersections(engine, df):
    if df is None or df.empty:
        intersection_df = []
    else:
        query = text('select DISTINCT day as key from sales')
        inner_df = pd.read_sql(query, engine)['key']
        inner_df = df['day'].unique()
        intersection_df = pd.merge(create_outer_df(df), pd.DataFrame({'key': inner_df}), on='key', how='inner')['key'].tolist()
        return intersection_df
    
# Function to remove intersections from the database
def delete_intersections(session, intersection_df):
    if intersection_df:
        delete_query = text('DELETE FROM sales WHERE day = ANY(:keys)')
        try:
            session.execute(delete_query, {'keys': intersection_df})
            session.commit()
        except Exception as e:
            print(f"Error while deleting records: {e}")

# Function to load data to database
def load_data_to_db(df, engine, name, IF_EXISTS):
    try:
        with engine.connect() as conn:
            df.to_sql(name, conn, if_exists=IF_EXISTS, index=False)
    except Exception as e:
        print(f"Error while loading data to the database: {e}")

# Function to transform and load dict data to database        
def transform_and_load_dict(engine, dfs):
    for df_name, df in dfs.items():
        df.columns = df.columns.str.lower()
        df.to_sql(df_name.lower(), engine, if_exists="replace", index=False)

# Main function
def main():
        
    # Create SSH tunnel
    with create_ssh_tunnel() as ssh_tunnel:
        
        # Create database engine
        engine = create_db_engine(ssh_tunnel)
        
        # Create session
        Session = sessionmaker(bind=engine)
    
        with Session() as session:
            # Iterate over dictionary items
            for table_name, table_info in DATA.items():  
                logging.info(f"Processing table: {table_name}")
            
                # Read Excel files
                df = read_excel_files(
                                    table_info["FOLDER_PATH_IN"], 
                                    table_info["FOLDER_PATH_OUT"], 
                                    table_info["SHEET"], 
                                    table_info["SKIP"], 
                                    table_info["COL_NAMES"]
                                    )

                # Process data
                df = process_data(df, table_info["COMPANIES"])
                                
                # Create intersections
                intersection_df = get_intersections(engine, df)
                
                # Remove intersections from the database
                delete_intersections(session, intersection_df)
                
                # Load data to database
                load_data_to_db(df, engine, table_name, table_info["IF_EXISTS"])   
                
                # Create Dict 
                dfs = load_excel_sheets(DICT_PATH, LIST_OF_SHEETS)
                
                #transform and load dict data to database
                transform_and_load_dict(engine, dfs)

if __name__ == '__main__':
    main()

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.9p1)
INFO:paramiko.transport:Authentication (password) successful!
INFO:root:Processing table: sales
INFO:root:Processing table: ms_sales
INFO:root:Processing table: ms_stock
